In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections

In [2]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
from collections import OrderedDict
from enum import Enum
import struct

from spinn_front_end_common.abstract_models import AbstractProvidesNKeysForPartition
from spinn_utilities.overrides import overrides
from pacman.executor.injection_decorator import inject_items
from pacman.model.graphs.machine import MachineVertex
from pacman.model.resources import ResourceContainer, VariableSDRAM
from pacman.utilities.utility_calls import is_single
from spinn_front_end_common.utilities.constants import (
    SYSTEM_BYTES_REQUIREMENT, BYTES_PER_WORD)
from spinn_front_end_common.utilities.exceptions import ConfigurationException
from spinn_front_end_common.utilities.helpful_functions import (
    locate_memory_region_for_placement)
from spinn_front_end_common.abstract_models.impl import (
    MachineDataSpecableVertex)
from spinn_front_end_common.interface.buffer_management.buffer_models import (
    AbstractReceiveBuffersToHost)
from spinn_front_end_common.interface.buffer_management import (
    recording_utilities)
from spinnaker_graph_front_end.utilities import SimulatorVertex
from spinnaker_graph_front_end.utilities.data_utils import (
    generate_system_data_region)

from data_specification.enums.data_type import DataType

from spinn_front_end_common.utilities import exceptions
import logging

logger = logging.getLogger(__name__)


class LatticeBasicCell(
    SimulatorVertex, MachineDataSpecableVertex,
    AbstractReceiveBuffersToHost,
    AbstractProvidesNKeysForPartition
):
    """ Cell which represents a cell within the 2d fabric
    """

    PARTITION_ID = "STATE"

    TRANSMISSION_DATA_SIZE = 2 * BYTES_PER_WORD  # has key and key
    STATE_DATA_SIZE = BYTES_PER_WORD  # 1 or 2 based off dead or alive
    # alive states, dead states
    NEIGHBOUR_INITIAL_STATES_SIZE = 2 * BYTES_PER_WORD
    RECORDING_ELEMENT_SIZE = STATE_DATA_SIZE  # A recording of the state

    POSITION_DATA_SIZE = 2 * BYTES_PER_WORD

    NEIGHBOUR_KEYS_SIZE = 9 * BYTES_PER_WORD  # for 8 directions and a mask

    VELOCITY_SIZE = 2 * BYTES_PER_WORD  # u_x and u_y

    VERTEX_INDEX_SIZE = BYTES_PER_WORD

    # The order of which directions are writeen to sdram
    ORDER_OF_DIRECTIONS = ["N", "W", "S", "E", "NW", "SW", "SE", "NE"]
    # Regions for populations
    DATA_REGIONS = Enum(
        value="DATA_REGIONS",
        names=[('SYSTEM', 0),
               ('TRANSMISSIONS', 1),
               ('POSITION', 2),
               ('NEIGHBOUR_KEYS', 3),
               ('VELOCITY', 4),
               ('VERTEX_INDEX', 5),
               ('RESULTS', 6)])

    def __init__(self, label, x_position, y_position, u_x, u_y):
        super(LatticeBasicCell, self).__init__(label, "lattice_cell.aplx")
        AbstractProvidesNKeysForPartition.__init__(self)
        # app specific data items
        self._x_position = x_position
        self._y_position = y_position
        # self._n_key = n_key
        # self._s_key = s_key
        # self._w_key = w_key
        # self._e_key = e_key
        # self._nw_key = nw_key
        # self._ne_key = ne_key
        # self._sw_key = sw_key
        # self._se_key = se_key
        self.u_x = u_x
        self.u_y = u_y

        self._loccation_vertices = OrderedDict()
        for direction in self.ORDER_OF_DIRECTIONS:
            self._loccation_vertices[direction] = None

    def set_direction_vertex(self, direction, vertex):
        """
        Add a vertex to the corresponding direction
        """
        self._loccation_vertices[direction] = vertex

    def _write_key_data(self, spec, routing_info, graph):
        spec.switch_write_focus(region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value)
        incoming_edges = graph.get_edges_ending_at_vertex_with_partition_name(self, self.PARTITION_ID)

        # verify the number of edges
        if len(incoming_edges) != 8:
            print(incoming_edges)
            raise exceptions.ConfigurationException("Should only have 8 edges")

        # get incoming edges
        for direction in self._loccation_vertices:
            key = routing_info.get_routing_info_from_pre_vertex(
                self._loccation_vertices[direction],
                self.PARTITION_ID).keys_and_masks[0].key
            if key is not None:
                spec.write_value(data=key)
            else:
                logger.warning("This lattice miss a edge from direction {}".format(direction))
                spec.write_value(data_type=DataType.INT32, data=-1)

        mask = routing_info.get_routing_info_from_pre_vertex(self._loccation_vertices["S"],
                                                             self.PARTITION_ID).keys_and_masks[0].mask
        spec.write_value(data=mask, data_type=DataType.UINT32)

    @inject_items({"data_n_time_steps": "DataNTimeSteps"})
    @overrides(
        MachineDataSpecableVertex.generate_machine_data_specification,
        additional_arguments={"data_n_time_steps"})
    def generate_machine_data_specification(
            self, spec, placement, machine_graph, routing_info, iptags,
            reverse_iptags, machine_time_step, time_scale_factor,
            data_n_time_steps):
        # Generate the system data region for simulation .c requirements
        generate_system_data_region(spec, self.DATA_REGIONS.SYSTEM.value,
                                    self, machine_time_step, time_scale_factor)

        # reserve memory regions
        spec.reserve_memory_region(
            region=self.DATA_REGIONS.TRANSMISSIONS.value,
            size=self.TRANSMISSION_DATA_SIZE, label="inputs")

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.POSITION.value,
            size=self.POSITION_DATA_SIZE, label="position"
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value,
            size=self.NEIGHBOUR_KEYS_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.VELOCITY.value,
            size=self.VELOCITY_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.VERTEX_INDEX.value,
            size=self.VERTEX_INDEX_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.RESULTS.value,
            size=recording_utilities.get_recording_header_size(1))

        # get recorded buffered regions sorted
        spec.switch_write_focus(self.DATA_REGIONS.RESULTS.value)
        spec.write_array(recording_utilities.get_recording_header_array(
            [self.RECORDING_ELEMENT_SIZE * data_n_time_steps]))

        # check got right number of keys and edges going into me
        partitions = \
            machine_graph.get_outgoing_edge_partitions_starting_at_vertex(self)
        if not is_single(partitions):
            raise ConfigurationException(
                "Can only handle one type of partition.")

        # check for duplicates
        edges = list(machine_graph.get_edges_ending_at_vertex(self))
        if len(edges) != 8:
            raise ConfigurationException(
                "I've not got the right number of connections. I have {} "
                "instead of 8".format(
                    len(machine_graph.get_edges_ending_at_vertex(self))))

        for edge in edges:
            if edge.pre_vertex == self:
                raise ConfigurationException(
                    "I'm connected to myself, this is deemed an error"
                    " please fix.")

        # write key needed to transmit with
        key = routing_info.get_first_key_from_pre_vertex(
            self, self.PARTITION_ID)

        spec.switch_write_focus(
            region=self.DATA_REGIONS.TRANSMISSIONS.value)
        spec.write_value(0 if key is None else 1)
        spec.write_value(0 if key is None else key)

        # write POSITION data
        spec.switch_write_focus(
            region=self.DATA_REGIONS.POSITION.value
        )
        spec.write_value(int(self._x_position))
        spec.write_value(int(self._y_position))

        # write neighbour keys
        # diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
        # spec.switch_write_focus(
        #     region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value
        # )
        # spec.write_value(self._n_key)
        # spec.write_value(self._w_key)
        # spec.write_value(self._s_key)
        # spec.write_value(self._e_key)
        # spec.write_value(self._nw_key)
        # spec.write_value(self._sw_key)
        # spec.write_value(self._se_key)
        # spec.write_value(self._ne_key)
        spec.switch_write_focus(region=self.DATA_REGIONS.VERTEX_INDEX.value)
        spec.write_value(machine_graph.vertices.index(self))

        # write the neighbour keys and masks
        self._write_key_data(spec, routing_info, machine_graph)

        spec.switch_write_focus(region=self.DATA_REGIONS.VELOCITY.value)
        spec.write_value(self.u_x, data_type=DataType.FLOAT_32)
        spec.write_value(self.u_y, data_type=DataType.FLOAT_32)

        # End-of-Spec:
        spec.end_specification()

    def get_data(self, buffer_manager, placement):
        # for buffering output info is taken form the buffer manager
        # get raw data, convert to list of booleans
        raw_data, data_missing = buffer_manager.get_data_by_placement(
            placement, 0)

        # do check for missing data
        if data_missing:
            print("missing_data from ({}, {}, {}); ".format(
                placement.x, placement.y, placement.p))

        # return the data, converted to list of booleans
        return [
            element
            for element in struct.unpack(
                "<{}f".format(len(raw_data) // BYTES_PER_WORD), raw_data)]

    @property
    @overrides(MachineVertex.resources_required)
    def resources_required(self):
        fixed_sdram = (SYSTEM_BYTES_REQUIREMENT + self.TRANSMISSION_DATA_SIZE +
                       self.POSITION_DATA_SIZE +
                       self.NEIGHBOUR_KEYS_SIZE +
                       self.VELOCITY_SIZE +
                       self.VERTEX_INDEX_SIZE +
                       recording_utilities.get_recording_header_size(1) +
                       recording_utilities.get_recording_data_constant_size(1))
        per_timestep_sdram = self.RECORDING_ELEMENT_SIZE
        return ResourceContainer(
            sdram=VariableSDRAM(fixed_sdram, per_timestep_sdram))

    @property
    def x_position(self):
        return self._x_position

    @overrides(AbstractProvidesNKeysForPartition.get_n_keys_for_partition)
    def get_n_keys_for_partition(self, partition, graph_mapper):
        return 8  # for its 8 neighbours to send

    @property
    def y_position(self):
        return self._y_position

    def __repr__(self):
        return self.label

    @overrides(AbstractReceiveBuffersToHost.get_recorded_region_ids)
    def get_recorded_region_ids(self):
        return [0]

    @overrides(AbstractReceiveBuffersToHost.get_recording_region_base_address)
    def get_recording_region_base_address(self, txrx, placement):
        return locate_memory_region_for_placement(
            placement, self.DATA_REGIONS.RESULTS.value, txrx)


In [3]:
from pacman.model.graphs.machine import MachineEdge


class LatticeEdge(MachineEdge):
    """
    Used for conjunction with a lattice
    """

    def __init__(self, pre_vertex, post_vertex, compass, label=None):
        MachineEdge.__init__(
            self, pre_vertex, post_vertex, label=label)
        self._compass = compass

    @property
    def compass(self):
        return self._compass

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        return "LatticeEdge: {}:{}".format(self._compass, self._label)

In [4]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
import math
import os
# from pacman.model.graphs.machine import MachineEdge

import spinnaker_graph_front_end as front_end

# 12000
runtime = 10000
time_step = 100000
MAX_X_SIZE_OF_FABRIC = 32
MAX_Y_SIZE_OF_FABRIC = 32
n_chips = (MAX_X_SIZE_OF_FABRIC * MAX_Y_SIZE_OF_FABRIC) // 15

ex = [0, 1, 0, -1, 0, 1, -1, -1, 1]
ey = [0, 0, 1, 0, -1, 1, 1, -1, -1]


def generatePositionMap(x_dim, y_dim):
    positionMap = {}
    #   diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
    for i in range(9):
        x_temp = int(x_dim - ex[i]) % MAX_X_SIZE_OF_FABRIC
        y_temp = int(y_dim - ey[i]) % MAX_Y_SIZE_OF_FABRIC
        positionMap[i] = (x_temp, y_temp)
    return positionMap


def generateKeys(position):
    return position[0] * MAX_X_SIZE_OF_FABRIC + position[1]


def initVelocity(x_pos, y_pos):
    U_0 = 0.01
    K = 30.0
    delta = 0.05
    x_temp = 1.0 * (x_pos) / MAX_X_SIZE_OF_FABRIC
    y_temp = 1.0 * (y_pos) / MAX_Y_SIZE_OF_FABRIC
    if y_temp <= 0.5:
        u_x = U_0 * math.tanh(K * (y_temp - 0.25))
    else:
        u_x = U_0 * math.tanh(K * (0.75 - y_temp))
    #     print("the u_x of ({}, {}) is {}".format(x_pos, y_pos, u_x))
    u_y = U_0 * delta * math.sin(2 * math.pi * (x_temp + 0.25))
    #     print("the u_y of ({}, {}) is {}".format(x_pos, y_pos, u_y))
    return u_x, u_y


# set up the front end and ask for the detected machines dimensions
front_end.setup(
    n_chips_required=n_chips, model_binary_folder=os.path.dirname(os.path.abspath("__file__")), machine_time_step=time_step,time_scale_factor=10)

# figure out if machine can handle simulation
cores = front_end.get_number_of_available_cores_on_machine()
# print(cores)
if cores <= (MAX_X_SIZE_OF_FABRIC * MAX_Y_SIZE_OF_FABRIC):
    raise KeyError("Don't have enough cores to run simulation")

# contain the vertices for the connection aspect
vertices = [
    [None for _ in range(MAX_X_SIZE_OF_FABRIC)]
    for _ in range(MAX_Y_SIZE_OF_FABRIC)]

# build vertices
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        u_x, u_y = initVelocity(x, y)
#         keymap = generatePositionMap(x, y)
        vert = LatticeBasicCell(
            "cell{}".format((x * MAX_X_SIZE_OF_FABRIC) + y),
            x, y, u_x, u_y)
        vertices[x][y] = vert
        front_end.add_machine_vertex_instance(vert)

# verify the initial state
output = ""
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        output += "({0}, {1}) ".format(vertices[x][y].x_position, vertices[x][y].y_position)
    output += "\n"
print(output)
print("\n\n")

# build edges
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
#         keymap = generatePositionMap(x, y)
        #   diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
        positions = [
            (x, (y + 1) % MAX_Y_SIZE_OF_FABRIC, "E"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC,
             (y + 1) % MAX_Y_SIZE_OF_FABRIC, "SE"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC, y, "S"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC,
             (y - 1) % MAX_Y_SIZE_OF_FABRIC, "SW"),
            (x, (y - 1) % MAX_Y_SIZE_OF_FABRIC, "W"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC,
             (y - 1) % MAX_Y_SIZE_OF_FABRIC, "NW"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC, y, "N"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC,
             (y + 1) % MAX_Y_SIZE_OF_FABRIC, "NE")]

         # build edges for each direction for this vertex
        for (dest_x, dest_y, compass) in positions:
            front_end.add_machine_edge_instance(LatticeEdge( vertices[x][y], vertices[dest_x][dest_y],compass, "edge between {} and {}".format(vertices[x][y], vertices[dest_x][dest_y])), LatticeBasicCell.PARTITION_ID)
            vertices[x][y].set_direction_vertex(direction=compass, vertex=vertices[dest_x][dest_y])

# run the simulation
front_end.run(runtime)

# get recorded data
recorded_data = dict()

# if not front_end.use_virtual_machine():
buffer_manager = front_end.buffer_manager()

# get the data per vertex
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        recorded_data[x, y] = vertices[x][y].get_data(
            front_end.buffer_manager(),
            front_end.placements().get_placement_of_vertex(
                vertices[x][y]))

# visualise it in text form (bad but no vis this time)
# for time in range(0, runtime*1000//time_step):
#     print("at time {}".format(time))
#     output = ""
#     for x in range(0, MAX_Y_SIZE_OF_FABRIC):
#         for y in range(0, MAX_Y_SIZE_OF_FABRIC):
#             output += "{} ".format(recorded_data[x, y][time])
#         output += "\n"
#     print(output)
#     print("\n\n")

# clear the machine
front_end.stop()


2020-07-17 00:03:01 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinnaker_graph_front_end/spiNNakerGraphFrontEnd.cfg, /home/spinnaker/.spiNNakerGraphFrontEnd.cfg
2020-07-17 00:03:01 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker
2020-07-17 00:03:01 WARNING: /home/spinnaker/reports has 34 old reports that have not been closed
2020-07-17 00:03:01 WARNING: /home/spinnaker/application_generated_data_files has 131 old reports that have not been closed
2020-07-17 00:03:01 INFO: Setting time scale factor to 10.
2020-07-17 00:03:01 INFO: Setting machine time step to 100000 micro-seconds.
Created spalloc job 5333821
2020-07-17 00:03:01 INFO: Created spalloc job 5333821
Waiting for board power commands to complete.
2020-07-17 00:03:01 INFO: Waiting

['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinnaker_graph_front_end/spiNNakerGraphFrontEnd.cfg', '/home/spinnaker/.spiNNakerGraphFrontEnd.cfg']


2020-07-17 00:03:06 INFO: Time 0:00:05.092854 taken by SpallocAllocator
2020-07-17 00:03:06 INFO: Creating transceiver for 10.11.197.17
2020-07-17 00:03:06 INFO: Working out if machine is booted
2020-07-17 00:03:10 INFO: Attempting to boot machine
2020-07-17 00:03:17 INFO: Found board with version [Version: SC&MP 3.2.5 at SpiNNaker:0:0:0 (built Thu Aug  1 08:15:06 2019)]
2020-07-17 00:03:17 INFO: Machine communication successful
2020-07-17 00:03:17 INFO: Detected a machine on IP address 10.11.197.17 which has 2562 cores and 384.0 links
2020-07-17 00:03:17 INFO: Time 0:00:11.261294 taken by MachineGenerator
Pre allocating resources for Extra Monitor support vertices
|0%                          50%                         100%|
2020-07-17 00:03:17 INFO: Time 0:00:00.149269 taken by PreAllocateResourcesForExtraMonitorSupport
2020-07-17 00:03:17 INFO: Starting execution process
2020-07-17 00:03:17 INFO: Time 0:00:00.014723 taken by NetworkSpecificationReport
Allocating virtual identifiers

(0, 0) (0, 1) (0, 2) (0, 3) (0, 4) (0, 5) (0, 6) (0, 7) (0, 8) (0, 9) (0, 10) (0, 11) (0, 12) (0, 13) (0, 14) (0, 15) (0, 16) (0, 17) (0, 18) (0, 19) (0, 20) (0, 21) (0, 22) (0, 23) (0, 24) (0, 25) (0, 26) (0, 27) (0, 28) (0, 29) (0, 30) (0, 31) 
(1, 0) (1, 1) (1, 2) (1, 3) (1, 4) (1, 5) (1, 6) (1, 7) (1, 8) (1, 9) (1, 10) (1, 11) (1, 12) (1, 13) (1, 14) (1, 15) (1, 16) (1, 17) (1, 18) (1, 19) (1, 20) (1, 21) (1, 22) (1, 23) (1, 24) (1, 25) (1, 26) (1, 27) (1, 28) (1, 29) (1, 30) (1, 31) 
(2, 0) (2, 1) (2, 2) (2, 3) (2, 4) (2, 5) (2, 6) (2, 7) (2, 8) (2, 9) (2, 10) (2, 11) (2, 12) (2, 13) (2, 14) (2, 15) (2, 16) (2, 17) (2, 18) (2, 19) (2, 20) (2, 21) (2, 22) (2, 23) (2, 24) (2, 25) (2, 26) (2, 27) (2, 28) (2, 29) (2, 30) (2, 31) 
(3, 0) (3, 1) (3, 2) (3, 3) (3, 4) (3, 5) (3, 6) (3, 7) (3, 8) (3, 9) (3, 10) (3, 11) (3, 12) (3, 13) (3, 14) (3, 15) (3, 16) (3, 17) (3, 18) (3, 19) (3, 20) (3, 21) (3, 22) (3, 23) (3, 24) (3, 25) (3, 26) (3, 27) (3, 28) (3, 29) (3, 30) (3, 31) 
(4, 0) (4, 1

2020-07-17 00:03:18 INFO: Time 0:00:00.231277 taken by MallocBasedChipIDAllocator
Inserting extra monitors into graphs
|0%                          50%                         100%|
2020-07-17 00:03:18 INFO: Time 0:00:00.049825 taken by InsertExtraMonitorVerticesToGraphs
Writing the board chip report
|0%                          50%                         100%|
2020-07-17 00:03:18 INFO: Time 0:00:00.009848 taken by BoardChipReport
Placing graph vertices
|0%                          50%                         100%|
2020-07-17 00:03:18 INFO: Time 0:00:00.253666 taken by RadialPlacer
Inserting edges between vertices which require FR speed up functionality.
|0%                          50%                         100%|
2020-07-17 00:03:18 INFO: Time 0:00:00.190061 taken by InsertEdgesToExtraMonitorFunctionality
Generating routing tables for data in system processes
|0%                          50%                         100%|
2020-07-17 00:03:18 INFO: Time 0:00:00.015806 taken by DataIn

2020-07-17 00:03:46 INFO: Time 0:00:00.433195 taken by SdramUsageReportPerChip
Updating run time
|0%                          50%                         100%|
2020-07-17 00:03:46 INFO: Time 0:00:00.281044 taken by ChipRuntimeUpdater
Getting provenance data from machine graph
|0%                          50%                         100%|
2020-07-17 00:03:46 INFO: Time 0:00:00.362375 taken by GraphProvenanceGatherer
2020-07-17 00:03:46 INFO: Time 0:00:00.000497 taken by DatabaseInterface
2020-07-17 00:03:46 INFO: ** Notifying external sources that the database is ready for reading **
2020-07-17 00:03:46 INFO: Time 0:00:00.001413 taken by NotificationProtocol
2020-07-17 00:03:46 INFO: *** Running simulation... *** 
Loading buffers
|0%                          50%                         100%|
2020-07-17 00:03:46 INFO: ** Awaiting for a response from an external source to state its ready for the simulation to start **
2020-07-17 00:03:46 INFO: ** Sending start / resume message to external

2020-07-17 00:07:16 WARNING: The extra monitor on 3, 1 has detected that 56 packets were dumped from a outgoing link of this chip's router. This often occurs when external devices are used in the script but not connected to the communication fabric correctly. These packets may have been reinjected multiple times and so this number may be a overestimate.
2020-07-17 00:07:16 WARNING: The extra monitor on 3, 2 has detected that 44 packets were dumped from a outgoing link of this chip's router. This often occurs when external devices are used in the script but not connected to the communication fabric correctly. These packets may have been reinjected multiple times and so this number may be a overestimate.
2020-07-17 00:07:16 WARNING: The extra monitor on 3, 3 has detected that 55 packets were dumped from a outgoing link of this chip's router. This often occurs when external devices are used in the script but not connected to the communication fabric correctly. These packets may have been 

2020-07-17 00:07:16 WARNING: The extra monitor on 6, 7 has detected that 35 packets were dumped from a outgoing link of this chip's router. This often occurs when external devices are used in the script but not connected to the communication fabric correctly. These packets may have been reinjected multiple times and so this number may be a overestimate.
2020-07-17 00:07:16 WARNING: The extra monitor on 7, 3 has detected that 2 packets were dumped from a outgoing link of this chip's router. This often occurs when external devices are used in the script but not connected to the communication fabric correctly. These packets may have been reinjected multiple times and so this number may be a overestimate.
2020-07-17 00:07:16 WARNING: The extra monitor on 7, 4 has detected that 20 packets were dumped from a outgoing link of this chip's router. This often occurs when external devices are used in the script but not connected to the communication fabric correctly. These packets may have been r

In [ ]:
for time in range(0, runtime*1000//time_step):
    print("at time {}".format(time))
    output = ""
    for x in range(0, MAX_Y_SIZE_OF_FABRIC):
        for y in range(0, MAX_Y_SIZE_OF_FABRIC):
            output += "{} ".format(recorded_data[x, y][time])
        output += "\n"
    print(output)
    print("\n\n")

In [ ]:
len(recorded_data[1,1])

In [ ]:
# visualise it in text form (bad but no vis this time)
u_x_output = collections.defaultdict(dict)
u_y_output = collections.defaultdict(dict)
for i in range(0, MAX_X_SIZE_OF_FABRIC):
    for j in range(0, MAX_Y_SIZE_OF_FABRIC):
        u_x_output[i][j] = recorded_data[i, j][8]
        u_y_output[i][j] = recorded_data[i, j][9]
u_x_output

In [ ]:
u_y_output

In [ ]:
voriticity = collections.defaultdict(dict)

for i in range(0, MAX_X_SIZE_OF_FABRIC):
    for j in range(0, MAX_Y_SIZE_OF_FABRIC): 
        voriticity[i][j] = 0.5 * (u_y_output[(i + 1) % MAX_X_SIZE_OF_FABRIC][j] - u_y_output[(i - 1)%MAX_X_SIZE_OF_FABRIC][j]) + 0.5 * (u_x_output[i][(j + 1)%MAX_Y_SIZE_OF_FABRIC] - u_x_output[i][(j - 1) % MAX_Y_SIZE_OF_FABRIC])

voriticity

In [ ]:
# Check the whole mass, if it keeo the same, then it is correct
for time in range(0,50):
    wholemass = 0
    for i in range(0, MAX_X_SIZE_OF_FABRIC):
        for j in range(0, MAX_Y_SIZE_OF_FABRIC): 
            wholemass+=recorded_data[i,j][time]
    print(wholemass)

In [ ]:
# Check hte momentum. if all 0 then it is correct
for time in range(50):
    wholeMomentum = 0
    for i in range(0, MAX_X_SIZE_OF_FABRIC):
        for j in range(0, MAX_Y_SIZE_OF_FABRIC): 
            wholeMomentum+=recorded_data[i,j][time]
    print(wholeMomentum)